# pyINSPECTA demo

In [ ]:
from __future__ import annotations

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams["figure.dpi"] = 150

from sdhdf import SDHDF
from sdhdf.logger import logger

logger.setLevel("INFO")

Let's load an SDHDF file - it is represented by an SDHDF class instance:

In [ ]:
# SDHDF v1.9.3 (NOTE: This file has a single integration only)
### Required to load local data for example - not needed for normal use
from importlib import resources

with resources.as_file(resources.files("sdhdf.data.tests")) as test_data:
    fname = "sdhdf_v1.9.3.hdf"
    file_path = test_data / fname

In [ ]:
sdhdf = SDHDF(file_path, parallel=True)

The SDHDF class instance contains information on the beams...

In [ ]:
sdhdf.beams[0]

...and all associated metadata:

In [ ]:
sdhdf.metadata

Let's explore the observation metadata - for example the primary_header dataset can be displayed with:

In [ ]:
sdhdf.metadata.primary_header

The attribute keys and descriptions can be accessed with:

In [ ]:
sdhdf.metadata.primary_header.attrs.keys()

In [ ]:
sdhdf.metadata.primary_header.attrs.values()

The attribute values, for example for the key 'TELESCOPE', can be accessed from the table:

In [ ]:
sdhdf.metadata.primary_header.table["TELESCOPE"][0]

The same applies for accessing the observation parameters with for example:

In [ ]:
sdhdf.metadata.obs_params

The Beam class contains the subband data - these can be accessed in a similar way to the beams:

In [ ]:
sdhdf.beams[0].subbands

The data are stored as either an `xarray.Dataset` or `pandas.DataFrame` depending on the data type:

In [ ]:
sdhdf.beams[0].subbands[0].astronomy_dataset

In [ ]:
sdhdf.metadata.frequency.table

For example, using the `xarray.DataArray` we have a lot of power to visualise and manipulate the data - see the [xarray documentation](https://docs.xarray.dev/en/stable/user-guide/data-structures.html) for more information.

I've also implemented some commonly-used commands on methods inside the dataclasses. For example, we can make a waterfall plot, plot a spectrum, and inspect the metadata. This is all done with `xarray` or `pandas` under the hood - so much more complex investigations are possible using those tools.

Here's a waterfall plot, which can be called from the `SDHDF` and `Beam` classes, but is really calling the base method on the `Subband` class:

In [ ]:
sdhdf.plot_waterfall(
    beam=0,
    subband=0,
    polarization=0,
    flag=False,
    norm=plt.cm.colors.LogNorm(vmin=1e0),
    y="ELAPSED_TIME",
)

Note that this is just a wrapper around the methods in `xarray`. You can use these directly for powerful visualisation and processing functionality, for example:

In [ ]:
if len(sdhdf.beams[0].subbands[0].astronomy_dataset.data["time"]) != 1:
    sdhdf.beams[0].subbands[0].astronomy_dataset.isel(polarization=0).data.plot(
        norm=plt.cm.colors.LogNorm(vmin=1e0, vmax=1e4), x="frequency", y="ELAPSED_TIME"
    )
else:
    logger.warning("Cannot create a waterfall plot with a single integration!")

We can also plot the spectrum, for example:

In [ ]:
ax = sdhdf.plot_spectrum(
    beam=0, subband=0, x="frequency", flag=False, color="black", linewidth=1
)
ax.set_yscale("log")

A wide-band plot can be called from the `SDHDF` object, but it also really just calling down to the `Beam` class:

In [ ]:
ax = sdhdf.plot_wide(
    beam=0, polarization=0, x="frequency", flag=False, color="black", linewidth=1
)
ax.set_yscale("log")

Now let's try and apply the RFI flagging routine:

In [ ]:
sdhdf.auto_flag_rfi(sigma=5, n_windows=1, flag_persistent=False)

In [ ]:
fig, ax = plt.subplots()
_ = sdhdf.plot_spectrum(
    beam=0,
    subband=0,
    polarization=0,
    x="frequency",
    flag=False,
    color="black",
    linewidth=1,
    alpha=0.2,
    ax=ax,
)
ax.set_yscale("log")

_ = sdhdf.plot_spectrum(
    beam=0,
    subband=0,
    polarization=0,
    x="frequency",
    flag=True,
    color="tab:red",
    linewidth=1,
    ax=ax,
)
ax.set_yscale("log")